# 📚 NNOptimized
This notebook builds an optimized neural network using regularization, dropout, and improved initialization.

In [1]:
# Import libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


In [9]:
# Load the dataset
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# Drop non-beneficial columns
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Group rare APPLICATION_TYPE values into 'Other'
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_df["APPLICATION_TYPE"].value_counts() < 500].index.tolist()
for app in application_types_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(app, "Other")

# Group rare CLASSIFICATION values into 'Other'
classification_counts = application_df["CLASSIFICATION"].value_counts()
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()
for cls in classifications_to_replace:
    application_df["CLASSIFICATION"] = application_df["CLASSIFICATION"].replace(cls, "Other")


In [4]:
# Encode categorical variables
application_df = pd.get_dummies(application_df)

# Split into features and target
X = application_df.drop("IS_SUCCESSFUL", axis=1)
y = application_df["IS_SUCCESSFUL"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [5]:
# Build the optimized neural network
nn = tf.keras.models.Sequential()

# First hidden layer with regularization
nn.add(tf.keras.layers.Dense(units=128, activation="relu", input_dim=X_train_scaled.shape[1],
                             kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.001)))

# Dropout for regularization
nn.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation="relu", kernel_initializer='he_normal'))
nn.add(tf.keras.layers.Dropout(0.3))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile model with customized optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
nn.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Model summary
nn.summary()


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         5,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,001 (62.50 KB)

 Trainable params: 16,001 (62.50 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Train the model
history = nn.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

# Evaluate the model
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.6829 - loss: 0.8165 - val_accuracy: 0.7282 - val_loss: 0.6629
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 851us/step - accuracy: 0.7209 - loss: 0.6634 - val_accuracy: 0.7300 - val_loss: 0.6175
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 831us/step - accuracy: 0.7279 - loss: 0.6181 - val_accuracy: 0.7272 - val_loss: 0.5936
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.7289 - loss: 0.5920 - val_accuracy: 0.7287 - val_loss: 0.5783
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step - accuracy: 0.7252 - loss: 0.5815 - val_accuracy: 0.7306 - val_loss: 0.5694
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 851us/step - accuracy: 0.7300 - loss: 0.5696 - val_accuracy: 0.7290 - val_loss: 0.5680
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 979us/step - accuracy: 0.7314 - loss: 0.5668 - val_accuracy: 0.7325 - val_loss: 0.5645
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 909us/step - accuracy: 0.7305 - loss: 0.5634 - va

In [8]:
# Save the optimized model
nn.save("AlphabetSoupCharity_Optimization.h5")
